In [1]:
import torch
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn as nn
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
torch.cuda.is_available()

True

In [3]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(y_test,y_pred,labels=None,lenth=20,ratio=False,cmap="Blues",annot=True,width=12,fmt="g"):
    if labels == None:
        labels = pd.Series(labels for labels in y_test.squeeze()).unique()
    conf_mat = pd.DataFrame(confusion_matrix(y_test.squeeze(), y_pred.squeeze(), labels=labels))
    if ratio == True:
        conf_mat = conf_mat.divide(conf_mat.sum(axis=1), axis=0)
    fig, ax = plt.subplots(figsize=(lenth, width))
    sns.heatmap(conf_mat,cmap=cmap,annot=annot,xticklabels=labels,yticklabels=labels,fmt=fmt)
    plt.ylabel("Actual")
    plt.xlabel("Predicted")
    plt.show()

In [4]:
f = open('word2idx.pkl','rb')
word2idx = pickle.load(f)
f.close()
f = open('idx2word.pkl','rb')
idx2word= pickle.load(f)
f.close()
f = open('word_df.pkl','rb')
word_df= pickle.load(f)
f.close()
f = open('word_embedding.pkl','rb')
word_embedding= pickle.load(f)
f.close()

In [5]:
EOS_NUM = 0
UNK_NUM = 1
PADDLING_NUM = 2

In [6]:
word_df['text'].apply(len).describe()

count    56821.000000
mean         9.087344
std          2.868494
min          0.000000
25%          7.000000
50%          9.000000
75%         11.000000
max         48.000000
Name: text, dtype: float64

In [7]:
word_df['text'].apply(len).describe()

count    56821.000000
mean         9.087344
std          2.868494
min          0.000000
25%          7.000000
50%          9.000000
75%         11.000000
max         48.000000
Name: text, dtype: float64

In [8]:
len(word_embedding)

24529

In [9]:
def getWordEmbedding(data):
    seq_embed = []
    _ = 0
    for word in data:
        seq_embed.append(word_embedding[word])
        _ += 1
        if _ == 9:
            break
    if _ <= 9:
        for i in range(9 - _):
            seq_embed.append(np.zeros(128))
    seq_embed = np.array(seq_embed)
    return seq_embed

In [10]:
# def getWordEmbedding(data):
#     seq_embed = []
#     _ = 0
#     for word in data:
#         seq_embed.append(word)
#         _ += 1
#         if _ == 9:
#             break
#     if _ <= 9:
#         for i in range(9 - _):
#             seq_embed.append()
#     seq_embed = np.array(seq_embed)
#     return seq_embed

In [11]:
word_df['text'] = word_df['text'].apply(getWordEmbedding)

In [12]:
train_df = word_df[['text','class']]
train_df['class'] = train_df['class'].apply(np.float32)

C:\Users\Javon\AppData\Local\Temp/ipykernel_1320/72822735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['class'] = train_df['class'].apply(np.float32)


In [13]:
train_df['text'][0]

array([[ 0.02666781,  0.17830361,  0.05896056, ..., -0.16031061,
         0.11336898, -0.08526121],
       [ 0.02924256,  0.19354033,  0.06433532, ..., -0.1772936 ,
         0.12500608, -0.09061292],
       [-0.0839004 , -1.2077056 , -0.2528606 , ..., -0.31072524,
         0.05834974,  1.3479041 ],
       ...,
       [ 0.03229726,  0.21405612,  0.07111906, ..., -0.19578084,
         0.13809216, -0.10043601],
       [ 0.02907891,  0.19331944,  0.06409359, ..., -0.17545098,
         0.12392033, -0.09148085],
       [ 0.03133297,  0.18446289,  0.06655352, ..., -0.22015649,
         0.14963819, -0.05733985]], dtype=float32)

In [14]:
X_train,X_test,y_train,y_test = train_test_split(
    train_df['text'].values,
    train_df['class'].values,
    test_size=0.2,
    stratify=train_df['class'],
    random_state=42,
)

In [15]:
class seqDataset(torch.utils.data.Dataset):
    def __init__(self,data,label,transformer = None):
        super(seqDataset,self).__init__()
        self.data = data
        self.label = label
        self.transformer = transformer
    
    def __getitem__(self,idx):
        if self.transformer:
            return transformer(self.data[idx]),self.label[idx]
        return self.data[idx],self.label[idx]
    
    def __len__(self):
        return len(self.data)

In [16]:
X_train = [np.array(i) for i in X_train]
X_test = [np.array(i) for i in X_test]

In [17]:
X_train[0]

array([[ 2.65811346e-02,  1.77673206e-01,  5.86651601e-02, ...,
        -1.59046233e-01,  1.12569943e-01, -8.53324458e-02],
       [ 3.62136737e-02,  2.37618491e-01,  7.96203986e-02, ...,
        -2.23189399e-01,  1.56768993e-01, -1.08234830e-01],
       [-2.75963939e-05,  1.87532933e-05,  1.98255293e-06, ...,
        -2.82110450e-05, -1.27364456e-05, -2.07734320e-05],
       ...,
       [-8.39003995e-02, -1.20770562e+00, -2.52860606e-01, ...,
        -3.10725242e-01,  5.83497398e-02,  1.34790409e+00],
       [ 3.55841517e-02,  2.31389880e-01,  7.78957680e-02, ...,
        -2.20986784e-01,  1.54726923e-01, -1.03786886e-01],
       [ 2.34421585e-02,  1.57128274e-01,  5.18420152e-02, ...,
        -1.40195757e-01,  9.92997736e-02, -7.57441595e-02]], dtype=float32)

In [18]:
train_dataset = seqDataset(X_train,y_train)
test_dataset = seqDataset(X_test,y_test)

In [19]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=500)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=500)

In [20]:
for i,(x,y) in enumerate(train_loader):
    print(x.shape)
    break

torch.Size([500, 9, 128])


In [21]:
def train(model,EPOCH=150,learning_rate=0.05):
    model = model
    opt = torch.optim.SGD(params=model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    EPOCH = EPOCH
    itr = 0
    model.cuda()
    for epoch in range(EPOCH):
        for i,(x,y) in enumerate(train_loader):
            batch_X = torch.FloatTensor(np.float32(x)).cuda()
            batch_Y = torch.LongTensor(np.int64(y)).cuda()
            output = model(batch_X)
            opt.zero_grad()
            loss = loss_fn(output,batch_Y)
            loss.backward()
            opt.step()
            if (i+1) % 20 == 0:
                itr += 20
                acc = (batch_Y.detach().cpu().numpy() == output.detach().cpu().argmax(axis=1).numpy()).mean()
                print('epoch\t:{}\titr:{}\tloss:{}\tacc:{}\n'.format(epoch,itr,loss,acc))
                # print(output.detach().cpu())
    return model

def test(model):
    accs = []
    y_pred = []
    y_true = []
    for i,(x,y) in enumerate(test_loader):
        batch_X = torch.FloatTensor(np.float32(x)).cuda()
        batch_Y = torch.LongTensor(np.int64(y)).cuda()
        output = model(batch_X)
        acc = (batch_Y.detach().cpu().numpy() == output.detach().cpu().argmax(axis=1).numpy()).mean()
        accs.append(acc)
        y_pred.extend(output.detach().cpu().argmax(axis=1).numpy())
        y_true.extend(batch_Y.detach().cpu().numpy()) 
    print(np.array(accs).mean())
    return y_pred,y_true

In [22]:
class SimpleRNN(nn.Module):
    def __init__(self,seq_len=9,embedding_size=128,h1_size=128,num_embeddings = len(idx2word)):
        super(SimpleRNN,self).__init__()
        self.embedding_size = embedding_size
        self.h1_size = h1_size
        self.seq_len = seq_len
        self.num_embeddings = num_embeddings

        self.embed = nn.Embedding(num_embeddings=self.num_embeddings,embedding_dim=self.embedding_size)

        self.layer1 = nn.Sequential(
            nn.RNN(input_size = self.embedding_size,hidden_size = self.h1_size,num_layers=2,batch_first=True),
        )

        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.seq_len*self.h1_size,self.h1_size),
            nn.Sigmoid(),
            nn.Linear(self.h1_size,10),
            nn.Softmax(),
        )
    def forward(self,x):
        # out = self.embed(x)
        out,hid = self.layer1(x)
        out = self.output(out)
        return out

In [23]:
class SimpleLSTM(nn.Module):
    def __init__(self,seq_len=9,embedding_size=128,h1_size=128,num_embeddings = len(idx2word)):
        super(SimpleLSTM,self).__init__()
        self.embedding_size = embedding_size
        self.h1_size = h1_size
        self.seq_len = seq_len
        self.num_embeddings = num_embeddings

        self.embed = nn.Embedding(num_embeddings=self.num_embeddings,embedding_dim=self.embedding_size)

        self.layer1 = nn.Sequential(
            nn.LSTM(input_size = self.embedding_size,hidden_size = self.h1_size,num_layers=2,batch_first=True),
        )

        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.seq_len*self.embedding_size,self.embedding_size),
            nn.Sigmoid(),
            nn.Linear(self.embedding_size,10),
            nn.Softmax(),
        )
    def forward(self,x):
        # out = self.embed(x)
        out,(hn, cn) = self.layer1(x)
        out = self.output(out)
        return out

In [24]:
class TransformerEncoder(nn.Module):
    def __init__(self,seq_len=9,embedding_size=128,num_embeddings = len(idx2word)):
        super(TransformerEncoder,self).__init__()
        self.num_embeddings = num_embeddings 
        self.embedding_size = embedding_size
        self.seq_len = seq_len
        self.emb = nn.Embedding(num_embeddings=self.num_embeddings,embedding_dim=self.embedding_size)
        self.layer1 = nn.Sequential(
            nn.TransformerEncoderLayer(d_model=embedding_size, nhead=8, batch_first=True),
            nn.TransformerEncoderLayer(d_model=embedding_size, nhead=8, batch_first=True),
            nn.TransformerEncoderLayer(d_model=embedding_size, nhead=8, batch_first=True),
            nn.TransformerEncoderLayer(d_model=embedding_size, nhead=8, batch_first=True),
        )

        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.seq_len*self.embedding_size,10),
            nn.Softmax(),
        )
    def forward(self,x):
        # out = self.emb(x)
        out = self.layer1(x)
        out = self.output(out)
        return out

In [25]:
class SimpleCNN(nn.Module):
    def __init__(self,embedding_size=128,num_embeddings = len(idx2word)):
        super(SimpleCNN,self).__init__()
        self.num_embeddings = num_embeddings
        self.embedding_size = embedding_size
        # self.embed = nn.Embedding(num_embeddings=self.num_embeddings,embedding_dim=self.embedding_size)
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels = 128,out_channels = 256,kernel_size=2),
            nn.ReLU(),
            nn.BatchNorm1d(256),
        )
        #[batch_size = 500,channels=128,seq_len - k_s + 1 = 8]
        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels = 256,out_channels = 256,kernel_size=2),
            nn.ReLU(),
            nn.BatchNorm1d(256),
        )
        #[batch_size = 500,embedding_size = 128,seq_len - k_s + 1 = 7]
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels = 256,out_channels = 256,kernel_size=3,stride = 2),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            # nn.MaxPool1d(6),
        )
        #[batch_size = 500,embedding_size = 256,(seq_len - k_s)/2+ 1 = 3]
        self.conv4 = nn.Sequential(
            nn.Conv1d(in_channels = 256,out_channels = 128,kernel_size = 3),
            nn.ReLU(),
            nn.BatchNorm1d(128),
        )
        #[batch_size = 500,embedding_size = 256,seq_len - k_s + 1 = 1]


        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128,10),
            nn.Softmax()
        )
    def forward(self,x):
        # out = self.embed(x).transpose(2,1)
        out = self.conv1(x.transpose(2,1))
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.output(out)
        return out

In [ ]:
model = train(TransformerEncoder(),EPOCH=500,learning_rate=0.5)
y_pred,y_true = test(model)
plot_confusion_matrix(np.array(y_true),np.array(y_pred))

In [ ]:
model = train(SimpleRNN(),EPOCH=500,learning_rate=0.5)
y_pred,y_true = test(model)
plot_confusion_matrix(np.array(y_true),np.array(y_pred))

In [ ]:
model = train(SimpleLSTM(),EPOCH=500,learning_rate=0.5)
y_pred,y_true = test(model)
plot_confusion_matrix(np.array(y_true),np.array(y_pred))

In [ ]:
model = train(SimpleCNN(),EPOCH=500,learning_rate=0.5)
y_pred,y_true = test(model)
plot_confusion_matrix(np.array(y_true),np.array(y_pred))